In [2]:
from __future__ import print_function
from pixell import enmap,utils, reproject, enplot
import numpy as np
import matplotlib.pyplot as plt
import os,sys
import urllib.request
from astropy.table import QTable
import astropy.units as u
from astropy.io import fits
import csv
import yaml
%matplotlib inline

RuntimeError: module compiled against API version 0xc but this version of numpy is 0xb

RuntimeError: module compiled against API version 0xc but this version of numpy is 0xb

In [3]:
def tileFinder(ra, dec, data):
    #Given an RA and Dec in deg, find the S18d tile containing that RA and Dec
    for i, tile in enumerate(data):
        box = tile['RADecSection']
        if box[0]<=ra<=box[1] and box[2]<=dec<= box[3]:
            return tile['tileName']
    return None

In [4]:
def s18dStamp(ra, dec, data, name, width = 0.5):
    #Find tile corresponding to RA, Dec
    path = '/scratch/r/rbond/jorlo/S18d_202006/filteredMaps/'
    tileName = tileFinder(ra, dec, data)
    if tileName == None: return None
    tile = enmap.read_map(path+tileName+'/Arnaud_M2e14_z0p4#'+tileName+'_filteredMap.fits')
    
    stamp = reproject.postage_stamp(tile, ra, dec, width*60, 0.5)
    
    return stamp


In [5]:
def s18dStack(ras, decs, data, width = 20., weight = False):
        stack = 0
        divisor = 0
        for i in range(len(ras)):
                path = '/scratch/r/rbond/jorlo/S18d_202006/filteredMaps/'
                tileName = tileFinder(ras[i], decs[i], data)
                
                if tileName is None: continue
                    
                tile = enmap.read_map(path+tileName+'/Arnaud_M2e14_z0p4#'+tileName+'_filteredMap.fits')
                stamp = reproject.postage_stamp(tile, ras[i], decs[i], width, 0.5)
                
                try: 
                    stack += stamp[0]
                except:
                    print(tileName)
                    print(tile)
                    print(stamp)

                divisor += 1

        
        try:
            stack /= divisor
        except: 
            print("Error: no items in stack")
            return None, None
        print("Number in stack: {}".format(divisor))
        return stack, divisor

In [6]:
with open('/scratch/r/rbond/jorlo/S18d_202006/selFn/tileDefinitions.yml') as f:
    
    s18d = yaml.load(f)


In [7]:
act = fits.open('/home/r/rbond/jorlo/dev/madcowsclusters/AdvACT_S18d_202006_MaDCoWSUnionMatch.fits')
in_act = act[1].data['MADCOWSUnion_name']

In [10]:
act[1].header

XTENSION= 'BINTABLE'           / binary table extension                         
BITPIX  =                    8 / array data type                                
NAXIS   =                    2 / number of array dimensions                     
NAXIS1  =                11600 / length of dimension 1                          
NAXIS2  =                   96 / length of dimension 2                          
PCOUNT  =                    0 / number of group parameters                     
GCOUNT  =                    1 / number of groups                               
TFIELDS =                  847 / number of table fields                         
TTYPE1  = 'name    '                                                            
TFORM1  = '19A     '                                                            
TTYPE2  = 'RADeg   '                                                            
TFORM2  = 'D       '                                                            
TTYPE3  = 'decDeg  '        